In [11]:
import os
os.getcwd()


'/content'

In [12]:
!git clone https://github.com/bchiusano/feathersInFocus.git


fatal: destination path 'feathersInFocus' already exists and is not an empty directory.


In [13]:
%cd feathersInFocus

/content/feathersInFocus


In [14]:
import sys
sys.path.insert(0, "/content/feathersInFocus/feathersInFocus")


In [15]:
!nvidia-smi

Fri Dec 12 10:27:15 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [16]:
import sys

print(sys.version)

3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]


In [17]:
import sys
print(sys.executable)
print(sys.version)


/usr/bin/python3
3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]


In [18]:
!python3 -m pip install skorch


In [19]:
!pwd

/content/feathersInFocus


In [20]:
!ls

attributes.npy	class_names.npy  test_images_path.csv	 train_images.csv
attributes.txt	feathersInFocus  test_images_sample.csv
baseline.py	test_images	 train_images


In [21]:
!git checkout model

Branch 'model' set up to track remote branch 'model' from 'origin'.
Switched to a new branch 'model'


In [22]:
import sys
sys.path.insert(0, "/content/feathersInFocus/feathersInFocus")


In [23]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from skorch import NeuralNetClassifier # skorch makes pytorch work with sklearn,
                                    # provides wrapper around PyTorch that has an sklearn interface
import torch
import torch.nn as nn
from pathlib import Path

from CNN import CNN
from DataLoad import DataLoad

In [ ]:
train_df = pd.read_csv('train_images.csv')
X = train_df['image_path'].values
y = train_df['label'].values

# skorch expect X, but dataset loads images from path
# so wrapping DataLoad makes it skorch-friendly
class BirdDataset(torch.utils.data.Dataset):
    def __init__(self,X,y):  
        df = pd.DataFrame({
            'image_path': X,
            'label': y
        })          
        self.dataset = DataLoad(data=df, data_path = Path('./train_images'))

    def __len__(self):
        return len(self.dataset)
    def __getitem__(self, index):
        return self.dataset[index]
    
# skorch
net = NeuralNetClassifier(
    module=CNN, 
    module__class_labels = 200,
    criterion=nn.CrossEntropyLoss,
    device='cpu',
    dataset = BirdDataset
)

# search space
#param_grid = {   THIS IS THE FIRST PARAM_GRID TO FIND LR, USED ADAM DID NOT USE BATCH SIZE
 #   'lr': [0.0001, 0.001],
  #  'max_epochs': [3], # should be more in real training
   # #'batch_size': [32, 64],
    #'optimizer': [torch.optim.AdamW], # try sgd optimizer 
    #'optimizer__weight_decay': [0.05]
#}
best_lr = 0.0001 # from initial param grid
param_grid = {
    'lr' : [best_lr],
    'max_epochs': [3], # 3
    'batch_size': [32,64],
    'optimizer': [torch.optim.AdamW],
    'optimizer__weight_decay':[0.01, 0.05]
}
grid = GridSearchCV(
    estimator=net,
    param_grid=param_grid,
    scoring='accuracy',
    cv=3,
    refit=True,
    verbose=2
)

grid.fit(X,y)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


  epoch    train_loss    valid_acc    valid_loss       dur
-------  ------------  -----------  ------------  --------
      1        5.4055       0.0115        5.2924  133.3779
      2        5.3060       0.0134        5.2834  131.2765
      3        5.3002       0.0153        5.2736  130.1465
[CV] END batch_size=32, lr=0.0001, max_epochs=3, optimizer=<class 'torch.optim.adamw.AdamW'>, optimizer__weight_decay=0.01; total time= 6.6min


/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py:960: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py", line 949, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_scorer.py", line 288, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_scorer.py", line 380, in _score
    y_pred = method_caller(
             ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_scorer.py", line 90, in _cached_call
   

  epoch    train_loss    valid_acc    valid_loss       dur
-------  ------------  -----------  ------------  --------
      1        5.4025       0.0076        5.2931  132.7451
      2        5.3070       0.0153        5.2864  131.7941
      3        5.3031       0.0153        5.2803  132.4079
[CV] END batch_size=32, lr=0.0001, max_epochs=3, optimizer=<class 'torch.optim.adamw.AdamW'>, optimizer__weight_decay=0.01; total time= 6.6min


...

/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py:960: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py", line 949, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_scorer.py", line 288, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_scorer.py", line 380, in _score
    y_pred = method_caller(
             ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_scorer.py", line 90, in _cached_call
   

  epoch    train_loss    valid_acc    valid_loss       dur
-------  ------------  -----------  ------------  --------
      1        5.4146       0.0115        5.2782  100.4113
      2        5.2918       0.0095        5.2717  99.9978
      3        5.2876       0.0095        5.2571  101.9217
[CV] END batch_size=64, lr=0.0001, max_epochs=3, optimizer=<class 'torch.optim.adamw.AdamW'>, optimizer__weight_decay=0.05; total time= 5.0min


/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py:960: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py", line 949, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_scorer.py", line 288, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_scorer.py", line 380, in _score
    y_pred = method_caller(
             ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_scorer.py", line 90, in _cached_call
   

  epoch    train_loss    valid_acc    valid_loss       dur
-------  ------------  -----------  ------------  --------
      1        5.4244       0.0076        5.2713  100.3196
      2        5.2961       0.0076        5.2593  101.3607
      3        5.2848       0.0076        5.2400  100.8029
[CV] END batch_size=64, lr=0.0001, max_epochs=3, optimizer=<class 'torch.optim.adamw.AdamW'>, optimizer__weight_decay=0.05; total time= 5.1min


/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py:960: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py", line 949, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_scorer.py", line 288, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_scorer.py", line 380, in _score
    y_pred = method_caller(
             ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_scorer.py", line 90, in _cached_call
   

In [1]:
print(grid.best_params_)

print(grid.best_score_)

NameError: name 'grid' is not defined